# **Estandarizacion de nombres de mascaras, dataset y demas:**
- Al aplicarle una mascara a un dataset seria conveniente primero definir la mascara y en otra linea de codigo aplicarsela al dataset. Esto para que podamos ver bien que le estamos aplicando y que condiciones tiene esa mascara.
- Ojo con utilizar mascaras o alguna otra variable, dataframe y demas, definida mucho mas arriba. No vaya a ser que le cambiemos el nombre por x motivo y nos ollvidamos que mas abajo la usamos para otra cosa y se rompa todo
- Los nombres para las cosas que sean en ingles, separando las palabras con " _ " y en lo posible que se pueda intuir mediante el nombre a que se refiere
- Como practica totalmente arbitraria intentar que los nombres no tengan mas de 5 palabras. ej: " mask_rooms_zeros " , " mask_rooms_surf_zeros " y asi
- Si le llamamos " mask " a algo, que realmente lo sea. No que sea ya una mascara aplicada a un dataset. Eso es el dataset data filtrado por la mascara
- A estos nuevos dataset llamemosle igual que la mascara que le aplicamos solo que reemplazamos el nombre mask por data. ej:  " mask_rooms_zeros "=  " data_rooms_zeros ";                   " data_rooms_surf_zeros " = " data_rooms_surf_zeros "
- Cuando creemos un nuevo dataset, un array, una lista, una serie, mascara o lo que sea poner primero el nombre del tipo de objeto y despues lo que hace, para saber lo que es eso
- Cuando hacemos una mascara para ver si algo es nulo o cero ponemos solo " null ". Es para no alargar los nombres



# Vemos como vamos aca 03/05

## Hasta aca realizamos
- Transformacion de todos los ceros a nulos
- Limpieza de filas nulas
- Eliminacion de columnas sin informacion relevante y con informacion redundante
- Eliminacion de state_name que representen menos del 1% del total de instancias
- Correccion en las variables surface_covered_in_m2 y surface_total_in_m2 en los casos que la primera era mayor que la segunda no siendo posible esto
- Imputacion de nulos en la columna place_name (si hacemos el paso del split de place_with_parent_names ese paso hay que eliminarlo)
- imputacion de datos de price_aprox_usd, surface_total_in_m2 y rooms mediante expresiones regulares de las columnas description y title
- Imputacion de datos en surface_covered_in_m2 y surface_total_in_m2 cuando uno era nulo pero el otro no
- Imputacion de las variables surface_total_in_m2, price_aprox_usd y price_usd_per_m2 mediante deduccion de otras variables conocidas

## Falta realizar
- Definir si se desechan las instancias que representen menos del 1% en funcion del state_name. Este es un problema de granulacion, ya que con la actual nos quedamos con muy pocas instancias para el modelo, pero aumentando la granulacion (haciendo una agregacion de todas estas en una nueva categoria, ej: resto del pais), no desechamos las instancias y solo perdemos granularidad pero no toda la informacion. **PREGUNTAR A FABRI**
- Corregir el patron con el que se busca el precio, para encontrar valores en pesos tambien, haciendo dos grupos, de esta manera el que matchee el grupo dolar se imputa directamente y el que matchee el grupo pesos imputarlo haciendo una conversion del valor. Por facilidad de codigo se imputa el primer grupo que matchea. **GERA , BENKOS Y ERICK**
- Sofisticar los patrones de las expresiones regulares para encontrar mas valores, especialmente en superficie. **GERA , BENKOS Y ERICK**
- La columna operation y country_name contienen una unica variable, en el caso de operation es sell y en el caso de country_name es argentina. No tiene sentido mantener las columnas siendo que no aportan informacion. Definir su tratamiento **PREGUNTAR A FABRI**
- Realizar una imputacion con media condicionada en rooms por superficie, y lugar. **GERA , BENKOS y ERICK** (falta la imputacion que no se bien como hacerla)
- Realizar una imputacion por media condicionada en superficie por media condicionada en rooms y lugar **GERA , BENKOS y ERICK** (falta la imputacion que no se bien como hacerla)
- Revisar si no hay otra forma de determinar el price y el price por metro cuadrado y si no lo hay tirar estas instancias que sean nulas **GERA , BENKOS y ERICK**
- Darle un tratamiento a expensas y floor. En propiedades tipo casas esta bien que sean nulas pero en departamentos no. Igualmente hay informacion ahi, quizas no sea bueno tirarla. *Como propuesta podriamos buscarlo con expresiones regulares en description y title ya que no tenemos de donde mas sacarla* *Tambien se podria ver la correlacion entre las expensas y el lugar y la superficie y si se encuentra una correlacion imputarlo por media condicionada por ejemplo **GERA , BENKOS y ERICK** **PREGUNTAR A FABRI**
- Darle un tratamiento a lat y lon y geonames id **JUEVES**
- Creacion de nuevas columnas estilo **Cercania a un espacio verde** **Cercania a un subte** **Mapa de delito** **GERA, BENKOS y ERICK**
- EDA: deteccion de outliers y representacion de datos y medidas estadisticas **PABLO Y ALE**
- Presentacion, formato revision de integridad de informacion **PABLO Y ALE**

# Importacion de librerias utilizadas

In [1]:
import pandas as pd
import numpy as np
import re

# Carga de la data de Properatti como un DataFrame de Pandas

In [2]:
data_location = "../Data/properatti.csv"

data = pd.read_csv(data_location, index_col=0)
data.head(3)

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...


# Renombro la columna lat-lon para evitar problemas

In [3]:
data.rename(columns={'lat-lon': 'lat_lon'}, inplace=True)

# Verificacion de instancias nulas

In [4]:
# Comprobamos la cantidad de filas donde tengan valor nulo en todas las columnas y las eliminamos  
data_clean_empty_row = data.dropna(axis = 0, how= 'all')
print(data.shape)
print(data_clean_empty_row.shape)
#En este caso no hay ninguna fila con todos los valores nulos

(121220, 25)
(121220, 25)


# Tratamiento de ceros en columnas

#### Dado que no hay ninguna variable que pueda tomar el valor cero, se van a reemplazar todos los valores ceros por NaN, para mejor tratamiento de los mismos

In [5]:
# Cantidad totales de ceros en el dataset
(data==0).sum()

operation                       0
property_type                   0
place_name                      0
place_with_parent_names         0
country_name                    0
state_name                      0
geonames_id                     0
lat_lon                         0
lat                             0
lon                             0
price                           1
currency                        0
price_aprox_local_currency      1
price_aprox_usd                 1
surface_total_in_m2           383
surface_covered_in_m2           2
price_usd_per_m2                0
price_per_m2                    0
floor                           0
rooms                           0
expenses                        0
properati_url                   0
description                     0
title                           0
image_thumbnail                 0
dtype: int64

In [6]:
# Cambio valores iguales a cero por NaN
data[data==0] = np.nan
(data==0).sum()

operation                     0
property_type                 0
place_name                    0
place_with_parent_names       0
country_name                  0
state_name                    0
geonames_id                   0
lat_lon                       0
lat                           0
lon                           0
price                         0
currency                      0
price_aprox_local_currency    0
price_aprox_usd               0
surface_total_in_m2           0
surface_covered_in_m2         0
price_usd_per_m2              0
price_per_m2                  0
floor                         0
rooms                         0
expenses                      0
properati_url                 0
description                   0
title                         0
image_thumbnail               0
dtype: int64

# Tratamiento de columnas con informacion no util o con informacion redundante

## Tratamiento de columnas *properati_url* y *image_thumbnail*

#### Vamos a desechar las columnas *properati_url* y *image_thumbnail* ya que no aportan informacion util

In [7]:
# desechamos las columnas properati_url y image_thumbnail de data
data = data.drop(['properati_url', 'image_thumbnail'], axis=1)
print("cantidad de columnas actuales=",data.shape[1])

cantidad de columnas actuales= 23


## Tratamiento de columnas *price* , *price_aprox_local_currency* y *price_aprox_usd* 

#### Vamos a quedarnos solo con la columna **price_aprox_usd** pudiendo desechar las columnas **price** y **price_aprox_local_currency** como asi tambien la columna **currency** ya que no aporta informacion adicional 
**Asumimos igual fiabilidad de los datos en las tres variables**

In [8]:
#Voy a verificar que no haya mas informacion en una columna que en las otras

#Hago tres mascaras para las variables de precios que contengan los nulos de las mismas
mask_price_null = data.price.isnull()
mask_price_aprox_local_currency_null = data.price_aprox_local_currency.isnull()
mask_price_aprox_usd_null = data.price_aprox_usd.isnull()

#Hago una interseccion de las tres mascaras
mask_prices_null = mask_price_null & mask_price_aprox_local_currency_null & mask_price_aprox_usd_null

#Si las tres mascaras tienen cada una la misma cantidad de elementos que la interseccion significa que los nulos se encuentran siempre en las mismas instancias
print("Cantidad de nulos en price= " , mask_price_null.sum())
print("cantidad de nulos en price_aprox_local_currency= " , mask_price_aprox_local_currency_null.sum())
print("Cantida de nulos en price_aprox_usd= " , mask_price_aprox_usd_null.sum())
print("Cantidad de nulos en la interseccion de las mascaras= " , mask_prices_null.sum())

#Devuelve un True si la cantidad de nulos es la misma en todas las mascaras por lo que la interseccion es perfecta
print(mask_price_null.sum()==mask_price_aprox_local_currency_null.sum()==mask_price_aprox_usd_null.sum()==mask_prices_null.sum())

Cantidad de nulos en price=  20411
cantidad de nulos en price_aprox_local_currency=  20411
Cantida de nulos en price_aprox_usd=  20411
Cantidad de nulos en la interseccion de las mascaras=  20411
True


In [9]:
#Desecho las columnas price, price_aprox_local_currency y currency
data = data.drop(['price', 'price_aprox_local_currency' , 'currency'], axis=1)
print("cantidad de columnas actuales= ",data.shape[1])

cantidad de columnas actuales=  20


## Tratamiento de columnas *price_usd_per_m2* y *price_per_m2*

#### Vamos a desechar la columna *price_per_m2* ya que es redundante con *price_usd_per_m2*
**Asumimos igual fiabilidad de datos en ambas variables**

In [10]:
#Voy a verificar que no haya mas informacion en una columna que en las otras

#Hago dos mascaras para las variables de precios por metro cuadrado que contengan los nulos de las mismas
mask_price_per_m2_null = data.price_per_m2.isnull()
mask_price_usd_per_m2_null = data.price_usd_per_m2.isnull()

#Hago una interseccion de las dos mascaras
mask_prices_per_m2_null = mask_price_per_m2_null & mask_price_usd_per_m2_null 

#Si las tres mascaras tienen cada una la misma cantidad de elementos que la interseccion significa que los nulos se encuentran siempre en las mismas instancias
print("Cantidad de nulos en price_per_m2= " , mask_price_per_m2_null.sum())
print("cantidad de nulos en price_usd_per_m2= " , mask_price_usd_per_m2_null.sum())
print("Cantidad de nulos en la interseccion de las mascaras= " , mask_prices_per_m2_null.sum())

#Devuelve un True si la cantidad de nulos es la misma en todas las mascaras por lo que la interseccion es perfecta
print(mask_price_per_m2_null.sum()==mask_price_usd_per_m2_null.sum()==mask_prices_per_m2_null.sum())

Cantidad de nulos en price_per_m2=  33562
cantidad de nulos en price_usd_per_m2=  52603
Cantidad de nulos en la interseccion de las mascaras=  28295
False


#### Como hay mas informacion en una columna que en la otra, veo en que instancias es nula la columna price_usd_per_m2 pero no lo es la columna price_per_m2 (solamente lo hago en esta ya que es la columna con la que nos vamos a quedar)

In [11]:
#Hago una mascara para ver cuando es nula la columna price_usd_per_m2 pero no lo es price_per_m2
mask_prices_per_m2 = data.price_usd_per_m2.isnull() & data.price_per_m2.notnull()
data_prices_per_m2 = data [mask_prices_per_m2]
data_prices_per_m2[["price_usd_per_m2" , "price_per_m2"]]

,price_usd_per_m2,price_per_m2
68,NaN,3263.888889
77,NaN,1764.705882
79,NaN,2809.523810
87,NaN,5147.058824
89,NaN,943.396226
...,...,...
121150,NaN,1826.923077
121151,NaN,1666.666667
121152,NaN,2257.575758
121155,NaN,1307.692308


In [12]:
#Realizo una imputacion en price_usd_per_m2 mediante price_per_m2

#Defino una variable que contenga el valor de conversion de pesos a dolar
pesos = 1/200

#Imputo en price_usd_per_m2 el valor de price_per_m2 multiplicado por el valor de la conversion de pesos a dolar
data.loc[data_prices_per_m2.index,"price_usd_per_m2"] = pesos *data.loc[data_prices_per_m2.index, "price_per_m2"]

In [13]:
#Desecho la columna price_per_m2
data = data.drop(['price_per_m2'], axis=1)
print("cantidad de columnas actuales= ",data.shape[1])

cantidad de columnas actuales=  19


## Tratamiento de columnas *lat-lon*, *lat* y *lon*

#### Vamos a desechar la columna *lat-lon* ya que es redundante con las columnas *lat* y *lon*
**Asumimos igual fiabilidad de datos en las tres variables**

In [14]:
#Voy a verificar que no haya mas informacion en una columna que en las otras

#Hago tres mascaras para las variables de latitud y longitud que contengan los nulos de las mismas
mask_lat_lon_null = data.lat_lon.isnull()
mask_lat_null = data.lat.isnull()
mask_lon_null = data.lon.isnull()

#Hago una interseccion de las tres mascaras
mask_inter_lat_lon_null = mask_lat_lon_null & mask_lat_null & mask_lon_null

#Si las tres mascaras tienen cada una la misma cantidad de elementos que la interseccion significa que los nulos se encuentran siempre en las mismas instancias
print("Cantidad de nulos en lat-lon= " , mask_lat_lon_null.sum())
print("cantidad de nulos en lat= " , mask_lat_null.sum())
print("Cantida de nulos en lon= " , mask_lon_null.sum())
print("Cantidad de nulos en la interseccion de las mascaras= " , mask_inter_lat_lon_null.sum())

#Devuelve un True si la cantidad de nulos es la misma en todas las mascaras por lo que la interseccion es perfecta
print(mask_lat_lon_null.sum()==mask_lat_null.sum()==mask_lon_null.sum()==mask_inter_lat_lon_null.sum())

Cantidad de nulos en lat-lon=  51550
cantidad de nulos en lat=  51550
Cantida de nulos en lon=  51550
Cantidad de nulos en la interseccion de las mascaras=  51550
True


In [15]:
#Desecho la columna lat_lon
data = data.drop(['lat_lon'], axis=1)
print("cantidad de columnas actuales= ",data.shape[1])

cantidad de columnas actuales=  18


# Limpieza de datos

# Analisis de instancias en funcion del state_name

In [16]:
# Vemos todos los diferentes state_name que hay en el DataSet
data.state_name.unique()

array(['Capital Federal', 'Bs.As. G.B.A. Zona Sur',
       'Buenos Aires Costa Atlántica', 'Entre Ríos',
       'Bs.As. G.B.A. Zona Norte', 'Santa Fe', 'Córdoba',
       'Bs.As. G.B.A. Zona Oeste', 'Misiones', 'Buenos Aires Interior',
       'Salta', 'Neuquén', 'Río Negro', 'San Luis', 'Mendoza',
       'Corrientes', 'Chubut', 'Tucumán', 'La Pampa', 'Chaco', 'San Juan',
       'Santa Cruz', 'Tierra Del Fuego', 'Catamarca',
       'Santiago Del Estero', 'Jujuy', 'La Rioja', 'Formosa'],
      dtype=object)

In [17]:
# Vemos la cantidad de instancias que tiene cada state_name
state_name_distribution = pd.value_counts(data.state_name)
state_name_distribution

Capital Federal                 32316
Bs.As. G.B.A. Zona Norte        25560
Bs.As. G.B.A. Zona Sur          13952
Córdoba                         12069
Santa Fe                        10172
Buenos Aires Costa Atlántica    10006
Bs.As. G.B.A. Zona Oeste         9322
Buenos Aires Interior            2291
Río Negro                         808
Neuquén                           733
Mendoza                           681
Tucumán                           674
Corrientes                        583
Misiones                          464
Entre Ríos                        369
Salta                             278
Chubut                            259
San Luis                          252
La Pampa                          157
Formosa                            65
Chaco                              57
San Juan                           40
Tierra Del Fuego                   31
Catamarca                          27
Jujuy                              26
Santa Cruz                         20
La Rioja    

In [18]:
# Vemos los porcentajes de la distribucion de los datos por provincia
total_data = data.shape[0]
percentage_state_name = 100*state_name_distribution/total_data
percentage_state_name

Capital Federal                 26.658967
Bs.As. G.B.A. Zona Norte        21.085629
Bs.As. G.B.A. Zona Sur          11.509652
Córdoba                          9.956278
Santa Fe                         8.391355
Buenos Aires Costa Atlántica     8.254413
Bs.As. G.B.A. Zona Oeste         7.690150
Buenos Aires Interior            1.889952
Río Negro                        0.666557
Neuquén                          0.604686
Mendoza                          0.561788
Tucumán                          0.556014
Corrientes                       0.480944
Misiones                         0.382775
Entre Ríos                       0.304405
Salta                            0.229335
Chubut                           0.213661
San Luis                         0.207886
La Pampa                         0.129517
Formosa                          0.053622
Chaco                            0.047022
San Juan                         0.032998
Tierra Del Fuego                 0.025573
Catamarca                        0

### Tomamos la decision de desechar las instancias correspondientes a los *state_name* que representan < 1% del total de instancias.

#### **Tomamos esta decision ya que consideramos que en estos state_names no contamos con las instancias suficientes como para realizar un analisis fiable**

In [19]:
# Tomamos la decision de quedarnos con los datos que son > 1%, que corresponden a Buenos Aires Interior paara arriba
mask_1_percent = percentage_state_name > 1
series_1_percent = percentage_state_name [mask_1_percent]
print(f'La muestra representa el {round(series_1_percent.sum(), 2)}% del total de los datos')
round(series_1_percent, 2)

La muestra representa el 95.44% del total de los datos


Capital Federal                 26.66
Bs.As. G.B.A. Zona Norte        21.09
Bs.As. G.B.A. Zona Sur          11.51
Córdoba                          9.96
Santa Fe                         8.39
Buenos Aires Costa Atlántica     8.25
Bs.As. G.B.A. Zona Oeste         7.69
Buenos Aires Interior            1.89
Name: state_name, dtype: float64

In [20]:
# Con la mascara generada anteriormente para quedarnos con los state_names mayores a 1%, eliminamos las instancias que no corresponden a los state_names seleccionados
data = data[data.state_name.isin(series_1_percent.index)]
data

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat,lon,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,floor,rooms,expenses,description,title
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,-34.661824,-58.508839,62000.0,55.0,40.0,1127.272727,NaN,NaN,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB
1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,-34.903883,-57.964330,150000.0,NaN,NaN,NaN,NaN,NaN,NaN,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,-34.652262,-58.522982,72000.0,55.0,55.0,1309.090909,NaN,NaN,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO
3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,-34.647797,-58.516424,95000.0,NaN,NaN,NaN,NaN,NaN,NaN,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado
4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,-38.002626,-57.549447,64000.0,35.0,35.0,1828.571429,NaN,NaN,NaN,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121215,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,NaN,NaN,870000.0,113.0,93.0,7699.115044,NaN,NaN,10000.0,TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...,Torre Forum Alcorta- Impecable 3 ambientes
121216,sell,house,Beccar,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,Argentina,Bs.As. G.B.A. Zona Norte,3436080.0,NaN,NaN,498000.0,360.0,360.0,1383.333333,NaN,NaN,NaN,Excelente e impecable casa en Venta en Las Lom...,Ruca Inmuebles | Venta | Lomas de San Isidro |...
121217,sell,apartment,Villa Urquiza,|Argentina|Capital Federal|Villa Urquiza|,Argentina,Capital Federal,3433775.0,-34.570639,-58.475596,131500.0,46.0,39.0,2858.695652,NaN,NaN,NaN,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...
121218,sell,apartment,Plaza Colón,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,NaN,NaN,NaN,95900.0,48.0,48.0,1997.916667,NaN,NaN,NaN,"2 Amb al contrafrente, luminoso. El departame...",2 amb. C/ dep. de servicio al contrafrente| Re...


## Desagregacion de la columna place_with_parent_names

**Aclaracion:** *Capital federal esta considerado como state_name, por lo que en city_name iran los barrios de la ciudad*

##### **Para realizar esto asumimos igual fiabilidad en los datos de las columnas place_name, country_name, state_name y place_with_parent_names**

In [21]:
# Realizo un nuevo dataset a partir de separar la columna place_with_parent_names en columnas separadas
data_place_with_parent_names = data["place_with_parent_names"].str.split("|", expand=True)
data_place_with_parent_names

,0,1,2,3,4,5,6
0,,Argentina,Capital Federal,Mataderos,,None,None
1,,Argentina,Bs.As. G.B.A. Zona Sur,La Plata,,None,None
2,,Argentina,Capital Federal,Mataderos,,None,None
3,,Argentina,Capital Federal,Liniers,,None,None
4,,Argentina,Buenos Aires Costa Atlántica,Mar del Plata,Centro,,None
...,...,...,...,...,...,...,...
121215,,Argentina,Capital Federal,Belgrano,,None,None
121216,,Argentina,Bs.As. G.B.A. Zona Norte,San Isidro,Beccar,,None
121217,,Argentina,Capital Federal,Villa Urquiza,,None,None
121218,,Argentina,Buenos Aires Costa Atlántica,Mar del Plata,Plaza Colón,,None


In [22]:
# Desecho las columnas 0, 5 y 6 ya que contienen informacion
data_place_with_parent_names = data_place_with_parent_names.drop([0,5,6], axis=1)

In [23]:
# renombro las columnas
data_place_with_parent_names.rename(columns={1: 'country_name' , 2 : 'state_name' , 3 : 'city_name' , 4 : 'neighborhood_name'}, inplace=True)
data_place_with_parent_names

,country_name,state_name,city_name,neighborhood_name
0,Argentina,Capital Federal,Mataderos,
1,Argentina,Bs.As. G.B.A. Zona Sur,La Plata,
2,Argentina,Capital Federal,Mataderos,
3,Argentina,Capital Federal,Liniers,
4,Argentina,Buenos Aires Costa Atlántica,Mar del Plata,Centro
...,...,...,...,...
121215,Argentina,Capital Federal,Belgrano,
121216,Argentina,Bs.As. G.B.A. Zona Norte,San Isidro,Beccar
121217,Argentina,Capital Federal,Villa Urquiza,
121218,Argentina,Buenos Aires Costa Atlántica,Mar del Plata,Plaza Colón


In [24]:
#Veo el total de strings vacios ""
print(data_place_with_parent_names.isnull().sum())
#Veo el total de nulos
print((data_place_with_parent_names=='').sum())

country_name            0
state_name              0
city_name               0
neighborhood_name    4422
dtype: int64
country_name             0
state_name               0
city_name             4422
neighborhood_name    71487
dtype: int64


In [25]:
# Transformo todas las cadenas vacias en city a nulos
data_city_name_empty = data_place_with_parent_names[data_place_with_parent_names.city_name==""]
data_place_with_parent_names.loc[data_city_name_empty.index,"city_name"] = None

In [26]:
# Transformo todas las cadenas vacias en city_name a nulos
data_neighborhood_name_empty = data_place_with_parent_names[data_place_with_parent_names.neighborhood_name==""]
data_place_with_parent_names.loc[data_neighborhood_name_empty.index,"neighborhood_name"] = None

In [27]:
#Veo el total de strings vacios ""
print(data_place_with_parent_names.isnull().sum())
#Veo el total de nulos
print((data_place_with_parent_names=='').sum())

country_name             0
state_name               0
city_name             4422
neighborhood_name    75909
dtype: int64
country_name         0
state_name           0
city_name            0
neighborhood_name    0
dtype: int64


In [28]:
#Elimino en data las columnas que voy a reemplazar
data = data.drop(["place_name","place_with_parent_names","country_name","state_name"], axis=1)

In [29]:
# Concateno data con data_place_with_parent_names. Lo realizamos con concat ya que tienen el mismo orden las instancias 
data = pd.concat([data, data_place_with_parent_names], axis=1)
data.columns

Index(['operation', 'property_type', 'geonames_id', 'lat', 'lon',
       'price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2',
       'price_usd_per_m2', 'floor', 'rooms', 'expenses', 'description',
       'title', 'country_name', 'state_name', 'city_name',
       'neighborhood_name'],
      dtype='object')

In [30]:
#Reordeno las columnas para que queden en el orden original
data= data[['operation', 'property_type','country_name', 'state_name', 'city_name', 
            'neighborhood_name', 'geonames_id', 'lat', 'lon','price_aprox_usd', 
            'surface_total_in_m2', 'surface_covered_in_m2',
       'price_usd_per_m2', 'floor', 'rooms', 'expenses', 'description',
       'title',]]

# Analis preliminar del DataSet

In [31]:
# Usamos dtypes para saber que tipo de datos se almacenan en el dataset
data.dtypes

operation                 object
property_type             object
country_name              object
state_name                object
city_name                 object
neighborhood_name         object
geonames_id              float64
lat                      float64
lon                      float64
price_aprox_usd          float64
surface_total_in_m2      float64
surface_covered_in_m2    float64
price_usd_per_m2         float64
floor                    float64
rooms                    float64
expenses                 float64
description               object
title                     object
dtype: object

In [32]:
#Vemos cuantos valores nulos tenemos por columna
is_null_result = data.isnull().sum()
print(is_null_result)

operation                     0
property_type                 0
country_name                  0
state_name                    0
city_name                  4422
neighborhood_name         75909
geonames_id               18527
lat                       48639
lon                       48639
price_aprox_usd           17871
surface_total_in_m2       37437
surface_covered_in_m2     17746
price_usd_per_m2          25390
floor                    108073
rooms                     69989
expenses                 101758
description                   2
title                         0
dtype: int64


In [33]:
#Vemos que porcentaje de nulos tiene cada columna
percentage_null = 100*is_null_result/data.shape[0]
percentage_null

operation                 0.000000
property_type             0.000000
country_name              0.000000
state_name                0.000000
city_name                 3.822350
neighborhood_name        65.615276
geonames_id              16.014626
lat                      42.043254
lon                      42.043254
price_aprox_usd          15.447583
surface_total_in_m2      32.360314
surface_covered_in_m2    15.339534
price_usd_per_m2         21.946961
floor                    93.417641
rooms                    60.498064
expenses                 87.958993
description               0.001729
title                     0.000000
dtype: float64

## Correccion de instancias que tienen surface_total_in_m2 < surface_covered_in_m2

In [34]:
# Vemos en que casos el valor surface_total_in_m2 es menor que el valor surface_covered_in_m2
mask_surface = data.surface_total_in_m2 < data.surface_covered_in_m2
data_surface = data[mask_surface]
#Le asigno en funcion del index de data_surface, los valores de surface_total_in_m2 a surface_covered_in_m2 y viceversa
data.loc[data_surface.index, "surface_total_in_m2"] = data_surface.loc[data_surface.index, "surface_covered_in_m2"]
data.loc[data_surface.index, "surface_covered_in_m2"] = data_surface.loc[data_surface.index, "surface_total_in_m2"]

# Obtencion de datos con expresiones regulares de las columnas *description* y *title*

## Obtencion de rooms con expresiones regulares

##### *Se obtienen los rooms nulos o iguales a cero en primera instancia con expresiones regulares. No se obtienen primero con una media condicionada de la superficie cubierta y el lugar ya que es mas preciso obtenerlos con un dato directo del dataset*

### Obtencion de rooms con expresion regular en *description*

In [35]:
# Aplicamos al DataSet una mascara booleana que contenga nulos de rooms
mask_rooms_null =  data.rooms.isnull()
data_rooms_null = data [mask_rooms_null]
series_rooms_null_description = data_rooms_null.description
series_rooms_null_description

0         2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...
1         Venta de departamento en décimo piso al frente...
2         2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...
3         PH 3 ambientes con patio. Hay 3 deptos en lote...
4         DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...
                                ...                        
121215    TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...
121216    Excelente e impecable casa en Venta en Las Lom...
121217    VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...
121218    2 Amb al contrafrente,  luminoso. El departame...
121219    Departamento de 77 m2 apto profesional. Edific...
Name: description, Length: 69989, dtype: object

In [36]:
# Usamos expresiones regulares para sacar los rooms en descripcion
# Creamos un patron que encuentra numeros de un digito o mas, que pueden o no tener un espacio antes de la combinacion de letras amb indistindo de mayusculas o minusculas
#logica del patron: que tome un numero de un digito que solamente tenga detras uno o mas espacios y seguido de uno o mas espacios y las letras amb
pattern = '(?P<rooms>\d+)\s+(?i)amb' 
pattern_rooms_regex = re.compile(pattern)

C:\Users\gtoso\AppData\Local\Temp\ipykernel_1460\3915632214.py:5: DeprecationWarning: Flags not at the start of the expression '(?P<rooms>\\d+)\\s+(?i' (truncated)
  pattern_rooms_regex = re.compile(pattern)


In [37]:
# Mostramos donde hay matches
description_rooms_result = series_rooms_null_description.apply(lambda x: pattern_rooms_regex.search(str(x)))
print(description_rooms_result)

0         <re.Match object; span=(0, 5), match='2 AMB'>
1                                                  None
2         <re.Match object; span=(0, 5), match='2 AMB'>
3         <re.Match object; span=(3, 8), match='3 amb'>
4                                                  None
                              ...                      
121215                                             None
121216                                             None
121217                                             None
121218    <re.Match object; span=(0, 5), match='2 Amb'>
121219                                             None
Name: description, Length: 69989, dtype: object


In [38]:
# Extraemos los datos que usaremos para la imputación (pongo que x sea cero cuando es none, ya que si me trae el valor None, no lo puedo pasar a float)
description_rooms_match = description_rooms_result.apply(lambda x: np.nan if x is None else x.group('rooms'))
print(description_rooms_match)

print("Rooms matcheados=",(description_rooms_match.notnull()).sum()) #aca veo que cantidad pudo matchear
print(description_rooms_match.unique()) #aca veo todos los rooms unicos que matcheo

0           2
1         NaN
2           2
3           3
4         NaN
         ... 
121215    NaN
121216    NaN
121217    NaN
121218      2
121219    NaN
Name: description, Length: 69989, dtype: object
Rooms matcheados= 18608
['2' nan '3' '1' '4' '7' '5' '6' '9' '2017' '103' '297' '072' '000' '11'
 '8' '085' '37' '800' '10' '12' '900' '0002' '400' '20174' '006' '17' '90'
 '0001' '772' '2004' '14' '24' '50' '20' '23' '65' '20173']


In [39]:
# Imputamos la serie rooms_match con su index en el DataSet 
data.loc[description_rooms_match.index, "rooms"] = description_rooms_match 
print(data[["property_type" , "rooms"]])
data[["rooms"]].dtypes

       property_type rooms
0                 PH     2
1          apartment   NaN
2          apartment     2
3                 PH     3
4          apartment   NaN
...              ...   ...
121215     apartment   NaN
121216         house   NaN
121217     apartment   NaN
121218     apartment     2
121219     apartment   NaN

[115688 rows x 2 columns]


rooms    object
dtype: object

### Obtencion de rooms con expresion regular en *title*

In [40]:
#Hacemos todo lo mismo que antes pero para obtener los rooms que se puedan de title
# Aplicamos al DataSet una mascara booleana que contenga cero rooms
mask_rooms_null = data.rooms.isnull()
data_rooms_null = data [mask_rooms_null]
series_rooms_null_title = data_rooms_null.title
series_rooms_null_title

1         VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37   ...
4         DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...
6         MUY BUEN PH AL FRENTE DOS DORMITORIOS , PATIO,...
7         JOSE HERNANDEZ 1400 MONOAMBIENTE  ESTRENAR CAT...
8         JOSE HERNANDEZ   1400 DOS AMBIENTES ESTRENAR ,...
                                ...                        
121214                                   Local - San Isidro
121215           Torre Forum Alcorta- Impecable 3 ambientes
121216    Ruca Inmuebles | Venta | Lomas de San Isidro |...
121217    VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...
121219           Departamento apto profesional en el Centro
Name: title, Length: 51381, dtype: object

In [41]:
title_rooms_result = series_rooms_null_title.apply(lambda x: pattern_rooms_regex.search(str(x)))
print(title_rooms_result)

1                                                    None
4          <re.Match object; span=(6, 11), match='2 AMB'>
6                                                    None
7                                                    None
8                                                    None
                               ...                       
121214                                               None
121215    <re.Match object; span=(31, 36), match='3 amb'>
121216                                               None
121217                                               None
121219                                               None
Name: title, Length: 51381, dtype: object


In [42]:
title_rooms_match = title_rooms_result.apply(lambda x: np.nan if x is None else x.group('rooms'))
print(title_rooms_match)

print("Rooms matcheados=",(title_rooms_match.notnull()).sum()) #aca veo que cantidad pudo matchear
print(title_rooms_match.unique()) #aca veo todos los rooms unicos que matcheo

1         NaN
4           2
6         NaN
7         NaN
8         NaN
         ... 
121214    NaN
121215      3
121216    NaN
121217    NaN
121219    NaN
Name: title, Length: 51381, dtype: object
Rooms matcheados= 2505
[nan '2' '6' '3' '4' '5' '9' '1' '7' '8' '15' '4000' '360' '12']


In [43]:
# Imputamos la serie rooms_match_title con su index en el DataSet 
data.loc[title_rooms_match.index, "rooms"] = title_rooms_match 
print(data[["property_type" , "rooms"]])
data[["rooms"]].dtypes

       property_type rooms
0                 PH     2
1          apartment   NaN
2          apartment     2
3                 PH     3
4          apartment     2
...              ...   ...
121215     apartment     3
121216         house   NaN
121217     apartment   NaN
121218     apartment     2
121219     apartment   NaN

[115688 rows x 2 columns]


rooms    object
dtype: object

In [44]:
 #Cambio el tipo de dato de object a float para homogeneizarlo con los datos numericos que tenemos
data["rooms"] = data["rooms"].astype("float")
data[["rooms"]].dtypes

rooms    float64
dtype: object

## Obtencion de surface_total_in_m2 con expresiones regulares

##### *Se obtienen la superficie en la que ambas superficies sean nulas y el precio y el precio por metro cuadrado es nulo o cero. De esta manera sabemos que no podemos imputar una superficie con la otra y no podemos inferir la superficie mediante el precio y el precio por metro cuadrado*

### Obtencion de surface_total_in_m2 con expresion regular en *description*

In [45]:
# Quiero ver que instancias son las que NO voy a poder imputar alguna superficie en funcion de otra u otras columnas de manera directa, por lo tanto lo tengo que obtener de expresiones regulares
# Aplicamos al DataSet una mascara booleana que contenga nulo de surface_total_in_m2 ademas de nulo price_usd_per_m2
surfaces_condition = data.surface_total_in_m2.isnull() & data.surface_covered_in_m2.isnull()
price_condition = data.price_aprox_usd.isnull()
price_per_m2_condition = data.price_usd_per_m2.isnull()

mask_surface_total_in_m2_null = surfaces_condition & (price_condition | price_per_m2_condition)
data_surface_total_in_m2_null = data [mask_surface_total_in_m2_null]

series_surface_total_in_m2_null_description = data_surface_total_in_m2_null.description
series_surface_total_in_m2_null_description

print("Cantidad de instancias en las que voy a buscar la superficie en title y descritpion= ",data_surface_total_in_m2_null.shape[0])

#Las instancias de data_prices_surfaces_prices_m2_null son las que tenemos que buscar en description y title buscando la superficie

Cantidad de instancias en las que voy a buscar la superficie en title y descritpion=  11437


##### *Tomamos la decision de imputar unicamente el primer matcheo y en la columna de superficie total*

In [46]:
# Usamos expresiones regulares para sacar la superficie en descripcion
# Creamos un patron que encuentra numeros de un digito o mas, que pueden o no tener un espacio antes de la combinacion de letras m2 indistindo de mayusculas o minusculas
pattern = '(?P<surface_total_in_m2>\d+)\s+(?i)m2' 
pattern_surface_total_in_m2_regex = re.compile(pattern)

C:\Users\gtoso\AppData\Local\Temp\ipykernel_1460\2861351483.py:4: DeprecationWarning: Flags not at the start of the expression '(?P<surface_total_in' (truncated)
  pattern_surface_total_in_m2_regex = re.compile(pattern)


In [47]:
# Mostramos donde hay matches
description_surface_total_in_m2_result = series_surface_total_in_m2_null_description.apply(lambda x: pattern_surface_total_in_m2_regex.search(str(x)))
print(description_rooms_result)

0         <re.Match object; span=(0, 5), match='2 AMB'>
1                                                  None
2         <re.Match object; span=(0, 5), match='2 AMB'>
3         <re.Match object; span=(3, 8), match='3 amb'>
4                                                  None
                              ...                      
121215                                             None
121216                                             None
121217                                             None
121218    <re.Match object; span=(0, 5), match='2 Amb'>
121219                                             None
Name: description, Length: 69989, dtype: object


In [48]:
description_surface_total_in_m2_match = description_surface_total_in_m2_result.apply(lambda x: np.nan if x is None else x.group('surface_total_in_m2'))
print(description_surface_total_in_m2_match)

print("surfaces matcheados=",(description_surface_total_in_m2_match.notnull()).sum()) #aca veo que cantidad pudo matchear

1         NaN
3         NaN
9         NaN
11        NaN
65        NaN
         ... 
121070    NaN
121079    NaN
121101    NaN
121134    124
121144    NaN
Name: description, Length: 11437, dtype: object
surfaces matcheados= 1235


In [49]:
# Imputamos la serie description_surface_total_in_m2_match con su index en el DataSet 
data.loc[description_surface_total_in_m2_match.index, "surface_total_in_m2"] = description_surface_total_in_m2_match 
print(data[["property_type" , "surface_total_in_m2"]])
data[["surface_total_in_m2"]].dtypes

       property_type surface_total_in_m2
0                 PH                  55
1          apartment                 NaN
2          apartment                  55
3                 PH                 NaN
4          apartment                  35
...              ...                 ...
121215     apartment                 113
121216         house                 360
121217     apartment                  46
121218     apartment                  48
121219     apartment                  77

[115688 rows x 2 columns]


surface_total_in_m2    object
dtype: object

### Obtencion de surface_total_in_m2 con expresion regular en *title*

In [50]:
# Realizo la misma mascara que para description
# Aplicamos al DataSet una mascara booleana que contenga nulo de surface_total_in_m2 ademas de nulo price_usd_per_m2
surfaces_condition = data.surface_total_in_m2.isnull() & data.surface_covered_in_m2.isnull()
price_condition = data.price_aprox_usd.isnull()
price_per_m2_condition = data.price_usd_per_m2.isnull()

mask_surface_total_in_m2_null = surfaces_condition & (price_condition | price_per_m2_condition)
data_surface_total_in_m2_null = data [mask_surface_total_in_m2_null]

series_surface_total_in_m2_null_title = data_surface_total_in_m2_null.title
series_surface_total_in_m2_null_title

1         VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37   ...
3                              PH 3 amb. cfte. reciclado   
9                WHITE 7637 - 2 DORMITORIOS CON PATIO      
11                                                  CHALET 
65                                 LOCAL COMERCIAL CON BAÑO
                                ...                        
121069                                   Casa - San Lorenzo
121070                      Barrio Sarmiento - Rondeau 200.
121079                             -Sarmiento- Rondeau 200.
121101    Casa a estrenar en San Matias, 3 dormitorios y...
121144    LA CASA DE TUS SUEÑOS!!! 3 DORM EN SANTINA NOR...
Name: title, Length: 10202, dtype: object

In [51]:
title_surface_total_in_m2_result = series_surface_total_in_m2_null_title.apply(lambda x: pattern_surface_total_in_m2_regex.search(str(x)))
print(title_surface_total_in_m2_result)

1         None
3         None
9         None
11        None
65        None
          ... 
121069    None
121070    None
121079    None
121101    None
121144    None
Name: title, Length: 10202, dtype: object


In [52]:
title_surface_total_in_m2_match = title_surface_total_in_m2_result.apply(lambda x: np.nan if x is None else x.group('surface_total_in_m2'))
print(title_surface_total_in_m2_match)

print("surfaces matcheados= ",(title_surface_total_in_m2_match.notnull()).sum()) #aca veo que cantidad pudo matchear

1         NaN
3         NaN
9         NaN
11        NaN
65        NaN
         ... 
121069    NaN
121070    NaN
121079    NaN
121101    NaN
121144    NaN
Name: title, Length: 10202, dtype: object
surfaces matcheados=  20


In [53]:
# Imputamos la serie description_surface_total_in_m2_match con su index en el DataSet 
data.loc[title_surface_total_in_m2_match.index, "surface_total_in_m2"] = title_surface_total_in_m2_match 
print(data[["property_type" , "surface_total_in_m2"]])
data[["surface_total_in_m2"]].dtypes

       property_type surface_total_in_m2
0                 PH                  55
1          apartment                 NaN
2          apartment                  55
3                 PH                 NaN
4          apartment                  35
...              ...                 ...
121215     apartment                 113
121216         house                 360
121217     apartment                  46
121218     apartment                  48
121219     apartment                  77

[115688 rows x 2 columns]


surface_total_in_m2    object
dtype: object

In [54]:
#Cambio el tipo de dato de object a float para homogeneizarlo con los datos numericos que tenemos
data["surface_total_in_m2"] = data["surface_total_in_m2"].astype("float")
data[["surface_total_in_m2"]].dtypes

surface_total_in_m2    float64
dtype: object

## Obtencion de price_aprox_usd mediante expresiones regulares

##### *Quiero obtener el price para las instancias en las que no tenemos price_aprox_usd y tampoco tenemos surface_total_in_m2 y surface_total_in_m2 o no tenemos price_usd_per_m2 y price_per_m2 *

### Obtencion de price con expresion regular en *description*

In [55]:
# Quiero ver que instancias son las que NO voy a poder imputar algun price en funcion de otra u otras columnas de manera directa, por lo tanto lo tengo que obtener de expresiones regulares

surface_condition = data.surface_total_in_m2.isnull() & data.surface_covered_in_m2.isnull()
price_condition = data.price_aprox_usd.isnull()
price_per_m2_condition = data.price_usd_per_m2.isnull()


mask_price_null = price_condition & (price_per_m2_condition | surfaces_condition)
data_price_null = data [mask_price_null]
series_price_null_description = data_price_null.description
series_price_null_description

data_price_null.shape[0]

#Las instancias de data_prices_surfaces_prices_m2_null son las que tenemos que buscar en description y title buscando el price

17871

In [56]:
# Usamos expresiones regulares para sacar la superficie en descripcion
# Creamos un patron que encuentra numeros de un digito o mas, que pueden o no tener un espacio antes de la combinacion de letras m2 indistindo de mayusculas o minusculas
pattern = '(?P<price_aprox_usd>\d+)\s+(?i)usd' 
pattern_price_regex = re.compile(pattern)

C:\Users\gtoso\AppData\Local\Temp\ipykernel_1460\4018430545.py:4: DeprecationWarning: Flags not at the start of the expression '(?P<price_aprox_usd>' (truncated)
  pattern_price_regex = re.compile(pattern)


In [57]:
# Mostramos donde hay matches
description_price_result = series_price_null_description.apply(lambda x: pattern_price_regex.search(str(x)))
print(description_price_result)

20        None
41        None
62        None
67        None
76        None
          ... 
121210    None
121211    None
121212    None
121213    None
121214    None
Name: description, Length: 17871, dtype: object


In [58]:
description_price_match = description_price_result.apply(lambda x: np.nan if x is None else x.group('price_aprox_usd'))
print(description_price_match)

print("price matcheados= ",(description_price_match.notnull()).sum()) #aca veo que cantidad pudo matchear

20        NaN
41        NaN
62        NaN
67        NaN
76        NaN
         ... 
121210    NaN
121211    NaN
121212    NaN
121213    NaN
121214    NaN
Name: description, Length: 17871, dtype: object
price matcheados=  11


In [59]:
# Imputamos la serie description_surface_total_in_m2_match con su index en el DataSet 
data.loc[description_price_match.index, "price_aprox_usd"] = description_price_match 
print(data[["property_type" , "price_aprox_usd"]])
data[["price_aprox_usd"]].dtypes

       property_type price_aprox_usd
0                 PH           62000
1          apartment          150000
2          apartment           72000
3                 PH           95000
4          apartment           64000
...              ...             ...
121215     apartment          870000
121216         house          498000
121217     apartment          131500
121218     apartment           95900
121219     apartment          129000

[115688 rows x 2 columns]


price_aprox_usd    object
dtype: object

### Obtencion de price con expresion regular en *title*

In [60]:
# Quiero ver que instancias son las que NO voy a poder imputar algun price en funcion de otra u otras columnas de manera directa, por lo tanto lo tengo que obtener de expresiones regulares

surface_condition = data.surface_total_in_m2.isnull() & data.surface_covered_in_m2.isnull()
price_condition = data.price_aprox_usd.isnull()
price_per_m2_condition = data.price_usd_per_m2.isnull()


mask_price_null = price_condition & (price_per_m2_condition | surfaces_condition)
data_price_null = data [mask_price_null]
series_price_null_title = data_price_null.title
series_price_null_title

data_price_null.shape[0]

#Las instancias de data_prices_surfaces_prices_m2_null son las que tenemos que buscar en title buscando el price

17860

In [61]:
title_price_result = series_price_null_title.apply(lambda x: pattern_price_regex.search(str(x)))
print(title_price_result)

20        None
41        None
62        None
67        None
76        None
          ... 
121210    None
121211    None
121212    None
121213    None
121214    None
Name: title, Length: 17860, dtype: object


In [62]:
title_price_match = title_price_result.apply(lambda x: np.nan if x is None else x.group('price_aprox_usd'))
print(title_price_match)

print("price matcheados= ",(title_price_match.notnull()).sum()) #aca veo que cantidad pudo matchear

20       NaN
41       NaN
62       NaN
67       NaN
76       NaN
          ..
121210   NaN
121211   NaN
121212   NaN
121213   NaN
121214   NaN
Name: title, Length: 17860, dtype: float64
price matcheados=  0


In [63]:
# Imputamos la serie description_surface_total_in_m2_match con su index en el DataSet 
data.loc[title_price_match.index, "price_aprox_usd"] = title_price_match 
print(data[["property_type" , "price_aprox_usd"]])
data[["price_aprox_usd"]].dtypes

       property_type price_aprox_usd
0                 PH           62000
1          apartment          150000
2          apartment           72000
3                 PH           95000
4          apartment           64000
...              ...             ...
121215     apartment          870000
121216         house          498000
121217     apartment          131500
121218     apartment           95900
121219     apartment          129000

[115688 rows x 2 columns]


price_aprox_usd    object
dtype: object

In [64]:
#Cambio el tipo de dato de object a float para homogeneizarlo con los datos numericos que tenemos
data["price_aprox_usd"] = data["price_aprox_usd"].astype("float")
data[["price_aprox_usd"]].dtypes

price_aprox_usd    float64
dtype: object

## Obtencion de floor mediante expresiones regulares

##### *Quiero obtener el floor para las instancias en las que es nulo

### Obtencion de floor con expresion regular en *description*

In [65]:
#Busno los nulos en floor
data_floor_null = data [data.floor.isnull()]

series_floor_null_description = data_floor_null.description
series_floor_null_description

data_floor_null.shape[0]



108073

In [66]:
# Usamos expresiones regulares para sacar el floor en la columna description
# Creamos un patron que encuentra numeros de un digito o mas, que pueden o no tener un espacio antes de la combinacion de letras piso indistindo de mayusculas o minusculas
pattern = '(?P<floor>\d+)\s+(?i)piso' 
pattern_floor_regex = re.compile(pattern)

C:\Users\gtoso\AppData\Local\Temp\ipykernel_1460\1837579789.py:4: DeprecationWarning: Flags not at the start of the expression '(?P<floor>\\d+)\\s+(?i' (truncated)
  pattern_floor_regex = re.compile(pattern)


In [67]:
# Mostramos donde hay matches
description_floor_result = series_floor_null_description.apply(lambda x: pattern_floor_regex.search(str(x)))
print(description_floor_result)

0         None
1         None
2         None
3         None
4         None
          ... 
121215    None
121216    None
121217    None
121218    None
121219    None
Name: description, Length: 108073, dtype: object


In [68]:
description_floor_match = description_floor_result.apply(lambda x: np.nan if x is None else x.group('floor'))
print(description_floor_match)

print("floor matcheados= ",(description_floor_match.notnull()).sum()) #aca veo que cantidad pudo matchear

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
121215    NaN
121216    NaN
121217    NaN
121218    NaN
121219    NaN
Name: description, Length: 108073, dtype: object
floor matcheados=  6842


In [69]:
# Imputamos la serie description_floor_match con su index en el DataSet 
data.loc[description_floor_match.index, "floor"] = description_floor_match 
print(data[["property_type" , "floor"]])
data[["floor"]].dtypes

       property_type floor
0                 PH   NaN
1          apartment   NaN
2          apartment   NaN
3                 PH   NaN
4          apartment   NaN
...              ...   ...
121215     apartment   NaN
121216         house   NaN
121217     apartment   NaN
121218     apartment   NaN
121219     apartment   NaN

[115688 rows x 2 columns]


floor    object
dtype: object

### Obtencion de floor con expresion regular en *title*

In [70]:
#Busno los nulos en floor
data_floor_null = data [data.floor.isnull()]

series_floor_null_title = data_floor_null.description
series_floor_null_title

data_floor_null.shape[0]

101231

In [71]:
# Mostramos donde hay matches
title_floor_result = series_floor_null_title.apply(lambda x: pattern_floor_regex.search(str(x)))
print(title_floor_result)

0         None
1         None
2         None
3         None
4         None
          ... 
121215    None
121216    None
121217    None
121218    None
121219    None
Name: description, Length: 101231, dtype: object


In [72]:
title_floor_match = title_floor_result.apply(lambda x: np.nan if x is None else x.group('floor'))
print(title_floor_match)

print("floor matcheados= ",(title_floor_match.notnull()).sum()) #aca veo que cantidad pudo matchear

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
121215   NaN
121216   NaN
121217   NaN
121218   NaN
121219   NaN
Name: description, Length: 101231, dtype: float64
floor matcheados=  0


In [73]:
# Imputamos la serie description_floor_match con su index en el DataSet 
data.loc[title_floor_match.index, "floor"] = title_floor_match 
print(data[["property_type" , "floor"]])
data[["floor"]].dtypes

       property_type floor
0                 PH   NaN
1          apartment   NaN
2          apartment   NaN
3                 PH   NaN
4          apartment   NaN
...              ...   ...
121215     apartment   NaN
121216         house   NaN
121217     apartment   NaN
121218     apartment   NaN
121219     apartment   NaN

[115688 rows x 2 columns]


floor    object
dtype: object

In [74]:
#Cambio el tipo de dato de object a float para homogeneizarlo con los datos numericos que tenemos
data["floor"] = data["floor"].astype("float")
data[["floor"]].dtypes

floor    float64
dtype: object

## Obtencion de expenses mediante expresiones regulares

##### *Quiero obtener el expensas para las instancias en las que es nulo

### Obtencion de floor con expresion regular en *description*

In [65]:
#Busno los nulos en floor
data_expenses_null = data [data.expenses.isnull()]

series_expenses_null_description = data_expenses_null.description
series_expenses_null_description

data_expenses_null.shape[0]



108073

In [66]:
# Usamos expresiones regulares para sacar el floor en la columna description
# Creamos un patron que encuentra numeros de un digito o mas, que pueden o no tener un espacio antes de la combinacion de letras piso indistindo de mayusculas o minusculas
pattern = '(?P<floor>\d+)\s+(?i)piso' 
pattern_floor_regex = re.compile(pattern)

C:\Users\gtoso\AppData\Local\Temp\ipykernel_1460\1837579789.py:4: DeprecationWarning: Flags not at the start of the expression '(?P<floor>\\d+)\\s+(?i' (truncated)
  pattern_floor_regex = re.compile(pattern)


In [67]:
# Mostramos donde hay matches
description_floor_result = series_floor_null_description.apply(lambda x: pattern_floor_regex.search(str(x)))
print(description_floor_result)

0         None
1         None
2         None
3         None
4         None
          ... 
121215    None
121216    None
121217    None
121218    None
121219    None
Name: description, Length: 108073, dtype: object


In [68]:
description_floor_match = description_floor_result.apply(lambda x: np.nan if x is None else x.group('floor'))
print(description_floor_match)

print("floor matcheados= ",(description_floor_match.notnull()).sum()) #aca veo que cantidad pudo matchear

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
121215    NaN
121216    NaN
121217    NaN
121218    NaN
121219    NaN
Name: description, Length: 108073, dtype: object
floor matcheados=  6842


In [69]:
# Imputamos la serie description_floor_match con su index en el DataSet 
data.loc[description_floor_match.index, "floor"] = description_floor_match 
print(data[["property_type" , "floor"]])
data[["floor"]].dtypes

       property_type floor
0                 PH   NaN
1          apartment   NaN
2          apartment   NaN
3                 PH   NaN
4          apartment   NaN
...              ...   ...
121215     apartment   NaN
121216         house   NaN
121217     apartment   NaN
121218     apartment   NaN
121219     apartment   NaN

[115688 rows x 2 columns]


floor    object
dtype: object

### Obtencion de floor con expresion regular en *title*

In [70]:
#Busno los nulos en floor
data_floor_null = data [data.floor.isnull()]

series_floor_null_title = data_floor_null.description
series_floor_null_title

data_floor_null.shape[0]

101231

In [71]:
# Mostramos donde hay matches
title_floor_result = series_floor_null_title.apply(lambda x: pattern_floor_regex.search(str(x)))
print(title_floor_result)

0         None
1         None
2         None
3         None
4         None
          ... 
121215    None
121216    None
121217    None
121218    None
121219    None
Name: description, Length: 101231, dtype: object


In [72]:
title_floor_match = title_floor_result.apply(lambda x: np.nan if x is None else x.group('floor'))
print(title_floor_match)

print("floor matcheados= ",(title_floor_match.notnull()).sum()) #aca veo que cantidad pudo matchear

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
121215   NaN
121216   NaN
121217   NaN
121218   NaN
121219   NaN
Name: description, Length: 101231, dtype: float64
floor matcheados=  0


In [73]:
# Imputamos la serie description_floor_match con su index en el DataSet 
data.loc[title_floor_match.index, "floor"] = title_floor_match 
print(data[["property_type" , "floor"]])
data[["floor"]].dtypes

       property_type floor
0                 PH   NaN
1          apartment   NaN
2          apartment   NaN
3                 PH   NaN
4          apartment   NaN
...              ...   ...
121215     apartment   NaN
121216         house   NaN
121217     apartment   NaN
121218     apartment   NaN
121219     apartment   NaN

[115688 rows x 2 columns]


floor    object
dtype: object

In [74]:
#Cambio el tipo de dato de object a float para homogeneizarlo con los datos numericos que tenemos
data["floor"] = data["floor"].astype("float")
data[["floor"]].dtypes

floor    float64
dtype: object

# Obtencion de datos mediante imputacion directa

## Obtencion de surface_total_in_m2 y surface_covered_in_m2 mediante imputacion directa

### Datos de Superficie total o superficie cubierta que quedaran nulos o ceros por ahora

In [75]:
# Realizamos una mascara para ver las instancias de superficie tanto cubierta como total de los cuales no tenemos para imputar un valor con el otro

mask_surfaces_null = data.surface_total_in_m2.isnull() & data.surface_covered_in_m2.isnull()
data_surfaces_null = data[mask_surfaces_null]

print (data_surfaces_null[["property_type" , "surface_total_in_m2" , "surface_covered_in_m2"]])

#En este DataFrame hay tanto superficies nulas como superficies iguales a cero

       property_type  surface_total_in_m2  surface_covered_in_m2
1          apartment                  NaN                    NaN
3                 PH                  NaN                    NaN
9              house                  NaN                    NaN
11             house                  NaN                    NaN
65             store                  NaN                    NaN
...              ...                  ...                    ...
121069         house                  NaN                    NaN
121070     apartment                  NaN                    NaN
121079     apartment                  NaN                    NaN
121101         house                  NaN                    NaN
121144         house                  NaN                    NaN

[10182 rows x 3 columns]


### Datos de superficie total o superficie que imputo uno con el otro

#### *Los imputo en este momento, para asegurarme de que cuando una es nula ambas lo son entonces le puedo dar un tratamiento mas sencillo a las condiciones booleanas mas adelante*

In [76]:
# Realizamos una mascara para ver que valores tenemos nulos de superficie total pero que la cubierta no es nula, por lo tanto se la podemos imputar
mask_surface_total_in_m2_null = data.surface_total_in_m2.isnull() & data.surface_covered_in_m2.notnull()
data_surface_total_in_m2_null = data[mask_surface_total_in_m2_null]

# Realizamos una mascara para ver que valores tenemos nulos de superficie cubierta pero que la total no es nula, por lo tanto se la podemos imputar
mask_surface_covered_in_m2_null =  data.surface_total_in_m2.notnull() & data.surface_covered_in_m2.isnull()
data_surface_covered_in_m2_null = data[mask_surface_covered_in_m2_null]


print (data_surface_total_in_m2_null[["property_type" , "surface_total_in_m2" , "surface_covered_in_m2"]])
print (data_surface_covered_in_m2_null[["property_type" , "surface_total_in_m2" , "surface_covered_in_m2"]])


       property_type  surface_total_in_m2  surface_covered_in_m2
68         apartment                  NaN                   72.0
77         apartment                  NaN                   85.0
79         apartment                  NaN                   42.0
87             house                  NaN                  170.0
89         apartment                  NaN                   53.0
...              ...                  ...                    ...
121155         house                  NaN                  130.0
121156         house                  NaN                  130.0
121207         house                  NaN                  178.0
121208         house                  NaN                  178.0
121210         house                  NaN                  120.0

[26000 rows x 3 columns]
       property_type  surface_total_in_m2  surface_covered_in_m2
10             house               1514.0                    NaN
15             house                164.0                    NaN

In [77]:
# Aca imputamos los valores nulos de superficie total con los valores de superficie cubierta, utilizando la mascara mask_surface_total_in_m2_null
data.loc[data_surface_total_in_m2_null.index, "surface_total_in_m2"] = data.loc[data_surface_total_in_m2_null.index, "surface_covered_in_m2"]

# Aca imputamos los valores nulos de superficie cubierta con los valores de superficie total, utilizando la mascara mask_surface_covered_in_m2_null
data.loc[data_surface_covered_in_m2_null.index, "surface_covered_in_m2"] = data.loc[data_surface_covered_in_m2_null.index, "surface_total_in_m2"]

### Obtención de las superficias nulas en funcion del precio y del precio por metro cuadrado

##### Vemos instancias que tengan el precio y el precio por metro cuadrado pero no la superficie

In [78]:
#Aca veo las instancias que tengan el precio y el precio por metro cuadrado pero no la superficie

In [79]:
# Se hacen tres mascaras booleanas: mask_surfaces_null = instancias sin datos de superficie / mask_prices_notnull = instancias con algun dato de precio / mask_prices_per_m2_notnull = instancias con algun dato de precio por metro cuadrado
condition_surfaces_null = data.surface_total_in_m2.isnull() & data.surface_covered_in_m2.isnull()
condition_price_notnull = data.price_aprox_usd.notnull()
condition_price_per_m2_notnull = data.price_usd_per_m2.notnull()

mask_surfaces_imput = condition_surfaces_null & condition_price_notnull & condition_price_per_m2_notnull
data_surfaces_imput = data[mask_surfaces_imput]

data_surfaces_imput[["price_aprox_usd" , "price_usd_per_m2" , "surface_total_in_m2" , "surface_covered_in_m2"]]

#Como da un dataframe vacio no hay ninguna instancia de superficie que pueda imputar de esta forma

,price_aprox_usd,price_usd_per_m2,surface_total_in_m2,surface_covered_in_m2


## Obtencion de price mediante imputacion directa en funcion de prices_per_m2 y surface

In [80]:
# Se hacen tres mascaras booleanas: mask_surfaces_nonull = instancias con datos de superficie no nulos / mask_prices_null = instancias sin datos de precios / mask_prices_per_m2_notnull = instancias con algun dato de precio por metro cuadrado
condition_surfaces_notnull = data.surface_total_in_m2.notnull() | data.surface_covered_in_m2.notnull()
condition_price_null =  data.price_aprox_usd.isnull()
condition_price_per_m2_notnull = data.price_usd_per_m2.notnull()

mask_prices_imput = condition_surfaces_null & condition_price_notnull & condition_price_per_m2_notnull
data_prices_imput = data[mask_prices_imput]

data_prices_imput[[ "price_aprox_usd" , "price_usd_per_m2" , "surface_total_in_m2" , "surface_covered_in_m2"]]

#Como da un dataframe vacio no hay ninguna instancia de price que pueda imputar de esta forma

,price_aprox_usd,price_usd_per_m2,surface_total_in_m2,surface_covered_in_m2


## Obtencion de price_per_m2 mediante imputacion directa en funcion de surface y price

In [81]:
# Se hacen tres mascaras booleanas: mask_surfaces_null = instancias sin datos de superficie / mask_prices_notnull = instancias con algun dato de precio / mask_prices_per_m2_notnull = instancias con algun dato de precio por metro cuadrado
condition_surfaces_notnull = data.surface_total_in_m2.notnull() | data.surface_covered_in_m2.notnull()
condition_price_notnull = data.price_aprox_usd.notnull()
condition_price_per_m2_null = data.price_usd_per_m2.isnull()

mask_prices_per_m2_imput = condition_surfaces_notnull & condition_price_notnull & condition_price_per_m2_null
data_prices_per_m2_imput = data[mask_prices_per_m2_imput]

data_prices_per_m2_imput[[ "price_aprox_usd" , "price_usd_per_m2" , "surface_total_in_m2" , "surface_covered_in_m2"]]

,price_aprox_usd,price_usd_per_m2,surface_total_in_m2,surface_covered_in_m2
675,130000.00,NaN,7.0,7.0
951,320000.00,NaN,1600.0,1600.0
952,450000.00,NaN,800.0,800.0
1017,36454.38,NaN,150.0,150.0
1402,150000.00,NaN,4.0,4.0
...,...,...,...,...
119502,235000.00,NaN,227.0,227.0
119823,23555.14,NaN,400.0,400.0
119835,360000.00,NaN,126.0,126.0
120122,198000.00,NaN,110.0,110.0


In [82]:
#Obtenemos el valor de price_usd_per_m2 haciendo price_aprox_usd sobre surface_total_in_m2
data.loc[data_prices_per_m2_imput.index,"price_usd_per_m2"] = data.loc[data_prices_per_m2_imput.index, "price_aprox_usd"] / data.loc[data_prices_per_m2_imput.index, "surface_total_in_m2"]

# Imputacion por medias condicionadas

### Imputamos las surface_covered_in_m2 y surface_total-in_m2 por medias condicionadas en funcion de las rooms, del state_name y del property_type

In [83]:
#Hago una mascara para ver las instancias con las que vamos a hacer el groupby, es decir, las instancias que tengan las rooms, el state_name , el property_type y la surface_covered_in_m2
data_surfaces_imput = data[ data.surface_total_in_m2.notnull() & data.rooms.notnull() & data.state_name.notnull() & data.property_type.notnull()]
data_surfaces_imput

,operation,property_type,country_name,state_name,city_name,neighborhood_name,geonames_id,lat,lon,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,floor,rooms,expenses,description,title
0,sell,PH,Argentina,Capital Federal,Mataderos,None,3430787.0,-34.661824,-58.508839,62000.0,55.0,40.0,1127.272727,NaN,2.0,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB
2,sell,apartment,Argentina,Capital Federal,Mataderos,None,3430787.0,-34.652262,-58.522982,72000.0,55.0,55.0,1309.090909,NaN,2.0,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO
4,sell,apartment,Argentina,Buenos Aires Costa Atlántica,Mar del Plata,Centro,3435548.0,-38.002626,-57.549447,64000.0,35.0,35.0,1828.571429,NaN,2.0,NaN,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...
12,sell,apartment,Argentina,Bs.As. G.B.A. Zona Norte,San Isidro,Martínez,3430813.0,-34.486019,-58.503814,115000.0,39.0,36.0,2948.717949,NaN,2.0,NaN,EXCELENTE DEPARTAMENTO 2 AMBIENTES CONTRAFRENT...,EXCELENTE DEPARTAMENTO 2 AMBIENTES CONTRAFRENT...
13,sell,apartment,Argentina,Capital Federal,Palermo,Palermo Soho,3430234.0,NaN,NaN,111700.0,50.0,30.0,2234.000000,NaN,1.0,NaN,Torre I Mondrian. 3 ambientes con terraza y d...,Vitraux Palermo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121138,sell,apartment,Argentina,Capital Federal,Saavedra,None,3429422.0,NaN,NaN,208000.0,69.0,60.0,3014.492754,NaN,2.0,2152.0,VENTA CON RENTA DEPARTAMENTO DE 2 AMBIENTES A ...,VENTA CON RENTA DEPARTAMENTO DE 2 AMBIENTES A ...
121139,sell,apartment,Argentina,Buenos Aires Costa Atlántica,Mar del Plata,Punta Iglesia,NaN,NaN,NaN,80523.0,37.0,37.0,2176.297297,18.0,3.0,NaN,"Condominio Royal Residence, espectacular empre...",Condominio a estrenar con amenities - Departam...
121153,sell,apartment,Argentina,Capital Federal,Barrio Norte,None,3436109.0,-34.599214,-58.402519,165000.0,91.0,65.0,1813.186813,NaN,3.0,NaN,EXCELENTE DEPARTAMENTO CON TERRAZA PROPIA Y PA...,EXCELENTE DEPARTAMENTO DE 3 AMBIENTES CON TERR...
121215,sell,apartment,Argentina,Capital Federal,Belgrano,None,3436077.0,NaN,NaN,870000.0,113.0,93.0,7699.115044,NaN,3.0,10000.0,TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...,Torre Forum Alcorta- Impecable 3 ambientes


In [84]:
#Hago un groupby que me traiga la media de surface_covered_in_m2 en funcion de property_type , state_name y rooms
data_surfaces_imput_groupby = data_surfaces_imput.groupby(by = ["property_type" , "state_name" ,"rooms" ])["surface_covered_in_m2"].mean()
data_surfaces_imput_groupby

property_type  state_name                rooms
PH             Bs.As. G.B.A. Zona Norte  1.0        69.444444
                                         2.0        51.106557
                                         3.0        77.573333
                                         4.0       100.468750
                                         5.0       133.470588
                                                     ...     
store          Córdoba                   5.0       120.000000
                                         6.0       480.000000
                                         14.0      400.000000
               Santa Fe                  2.0        20.000000
                                         8.0      1027.000000
Name: surface_covered_in_m2, Length: 331, dtype: float64

In [85]:
#Aca hago la imputacion en las instancias correspondientes

### Imputamos las rooms por medias condicionadas en funcion de la surface_covered_in_m2 y del state_name

##### Primero discretizo en cuatro categorias a la columna surface_covered_in_m2

In [86]:
#Con qcut discretizo la columa surface_covered_in_m2
serie_surfaces_category = pd.qcut(data.surface_covered_in_m2, 4,labels = ["chico","mediano","grande","muy_grande"])
serie_surfaces_category

0              chico
1                NaN
2            mediano
3                NaN
4              chico
             ...    
121215        grande
121216    muy_grande
121217         chico
121218       mediano
121219        grande
Name: surface_covered_in_m2, Length: 115688, dtype: category
Categories (4, object): ['chico' < 'mediano' < 'grande' < 'muy_grande']

In [87]:
#Creamos un dataset auxiliar y la instrtamos la serie obtenida anteriormente
data_aux = data
data_aux.insert(12,"surface_category",serie_surfaces_category)
data_aux.head(3)

,operation,property_type,country_name,state_name,city_name,neighborhood_name,geonames_id,lat,lon,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,surface_category,price_usd_per_m2,floor,rooms,expenses,description,title
0,sell,PH,Argentina,Capital Federal,Mataderos,None,3430787.0,-34.661824,-58.508839,62000.0,55.0,40.0,chico,1127.272727,NaN,2.0,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB
1,sell,apartment,Argentina,Bs.As. G.B.A. Zona Sur,La Plata,None,3432039.0,-34.903883,-57.964330,150000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...
2,sell,apartment,Argentina,Capital Federal,Mataderos,None,3430787.0,-34.652262,-58.522982,72000.0,55.0,55.0,mediano,1309.090909,NaN,2.0,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO


In [88]:
#Hago una mascara para ver las instancias con las que vamos a hacer el groupby, es decir, las instancias que tengan la surface_covered_in_m2 y las rooms
data_rooms_imput = data_aux[data_aux.rooms.notnull() & data_aux.surface_covered_in_m2.notnull()]
data_rooms_imput.head(3)

,operation,property_type,country_name,state_name,city_name,neighborhood_name,geonames_id,lat,lon,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,surface_category,price_usd_per_m2,floor,rooms,expenses,description,title
0,sell,PH,Argentina,Capital Federal,Mataderos,None,3430787.0,-34.661824,-58.508839,62000.0,55.0,40.0,chico,1127.272727,NaN,2.0,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB
2,sell,apartment,Argentina,Capital Federal,Mataderos,None,3430787.0,-34.652262,-58.522982,72000.0,55.0,55.0,mediano,1309.090909,NaN,2.0,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO
4,sell,apartment,Argentina,Buenos Aires Costa Atlántica,Mar del Plata,Centro,3435548.0,-38.002626,-57.549447,64000.0,35.0,35.0,chico,1828.571429,NaN,2.0,NaN,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...


In [89]:
data_rooms_imput_groupby = data_rooms_imput.groupby(by = ["property_type" , "state_name" , "surface_category" ])["rooms"].mean()
data_rooms_imput_groupby

property_type  state_name                surface_category
PH             Bs.As. G.B.A. Zona Norte  chico               2.623853
                                         mediano             2.795122
                                         grande              3.477966
                                         muy_grande          4.578947
               Bs.As. G.B.A. Zona Oeste  chico               2.591667
                                                               ...   
store          Córdoba                   muy_grande          4.833333
               Santa Fe                  chico               2.000000
                                         mediano                  NaN
                                         grande                   NaN
                                         muy_grande          8.000000
Name: rooms, Length: 128, dtype: float64

In [90]:
# Aca debo imputarlo